In [ ]:
pip install pandas transformers torch numpy openpyxl


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
import torch
import numpy as np

# Function to load Excel data
def load_excel_data(file_path):
    df = pd.read_excel("/content/traningdata.xlsx")
    return df

# Initialize GPT-2 model and tokenizer
def initialize_gpt2_model():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')
    return tokenizer, model

# Function to generate embeddings for a given text
def generate_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)  # Tokenize input text
    outputs = model(**inputs)  # Get the output from the model
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()  # Mean pooling to get fixed-size embedding
    return embedding

# Function to process Excel data and generate embeddings for each row
def generate_embeddings_for_excel(df, tokenizer, model):
    embeddings = []
    for _, row in df.iterrows():
        text_representation = ' '.join(map(str, row.values))  # Combine all columns into a single string
        embedding = generate_embedding(text_representation, tokenizer, model)
        embeddings.append(embedding)
    return np.array(embeddings)

# Function to save embeddings to a file (NumPy format or CSV)
def save_embeddings(embeddings, output_path, file_format='npy'):
    if file_format == 'npy':
        np.save(output_path, embeddings)  # Save as NumPy binary file
    elif file_format == 'csv':
        embeddings_flat = [emb.flatten() for emb in embeddings]
        emb_df = pd.DataFrame(embeddings_flat)
        emb_df.to_csv(output_path, index=False)

# Main function to handle the flow
def main():
    file_path = "/content/traningdata.xlsx"  # Replace with your Excel file path
    output_path = "embeddings.csv"  # Output file name
    file_format = 'csv'  # Choose 'npy' or 'csv'

    # Load Excel data
    df = load_excel_data(file_path)

    # Initialize GPT-2 model and tokenizer
    tokenizer, model = initialize_gpt2_model()

    # Generate embeddings for the Excel data
    embeddings = generate_embeddings_for_excel(df, tokenizer, model)

    # Save embeddings to a file
    save_embeddings(embeddings, output_path, file_format)

if __name__ == "__main__":
    main()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
##################################################
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
import torch
import numpy as np

# Function to load Excel data
def load_excel_data(file_path):
    df = pd.read_excel(file_path)
    return df

# Initialize GPT-2 model and tokenizer
def initialize_gpt2_model():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')
    return tokenizer, model

# Function to generate embeddings for a given text
def generate_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)  # Tokenize input text
    outputs = model(**inputs)  # Get the output from the model
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()  # Mean pooling to get fixed-size embedding
    return embedding

# Function to process Excel data and generate embeddings for each row
def generate_embeddings_for_excel(df, tokenizer, model, target_column):
    embeddings = []
    targets = df[target_column].values  # Extract target column values
    for _, row in df.drop(columns=[target_column]).iterrows():  # Drop target column for embedding
        text_representation = ' '.join(map(str, row.values))  # Combine all columns into a single string
        embedding = generate_embedding(text_representation, tokenizer, model)
        embeddings.append(embedding)
    return np.array(embeddings), targets

# Function to save embeddings to an Excel file
def save_embeddings_to_excel(embeddings, targets, output_path):
    embeddings_flat = [emb.flatten() for emb in embeddings]
    emb_df = pd.DataFrame(embeddings_flat)
    emb_df['Class'] = targets  # Add target column to the DataFrame
    emb_df.to_excel(output_path, index=False)  # Save as Excel file

# Main function to handle the flow
def main():
    file_path = "traningdata.xlsx"  # Replace with your Excel file path
    output_path = "embeddings_with_class.xlsx"  # Output file name
    target_column = "Class"  # Define your target column

    # Load Excel data
    df = load_excel_data(file_path)

    # Initialize GPT-2 model and tokenizer
    tokenizer, model = initialize_gpt2_model()

    # Generate embeddings for the Excel data
    embeddings, targets = generate_embeddings_for_excel(df, tokenizer, model, target_column)

    # Save embeddings and target to an Excel file
    save_embeddings_to_excel(embeddings, targets, output_path)

if __name__ == "__main__":
    main()
